<a href="https://colab.research.google.com/github/rdukewiesenb/Projects/blob/master/UnknownOfficerData/OfficerX2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
import pandas as pd
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt
from scipy import stats
import plotly.io as pio

In [9]:
from google.colab import files
uploaded = files.upload()

Saving yg821jf8611_sc_statewide_2020_04_01.csv.zip to yg821jf8611_sc_statewide_2020_04_01.csv.zip


In [1]:
!unzip yg821jf8611_sc_statewide_2020_04_01.csv.zip

Archive:  yg821jf8611_sc_statewide_2020_04_01.csv.zip
replace sc_statewide_2020_04_01.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
df = pd.read_csv('sc_statewide_2020_04_01.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.



In [7]:
print(df.shape)
df.head()

(8983810, 34)


,raw_row_number,date,location,lat,lng,county_name,subject_age,subject_race,subject_sex,officer_id_hash,officer_age,officer_race,department_id,type,violation,arrest_made,citation_issued,outcome,contraband_found,contraband_drugs,contraband_weapons,contraband_alcohol,contraband_other,search_conducted,search_person,search_vehicle,reason_for_stop,raw_contact_type,raw_sex,raw_race,raw_sectionnum,raw_offensecode,raw_contrabanddesc,raw_officer_race
0,1,2006-02-12,US Prmary 17,33.19950,-79.39269,Georgetown County,33.0,white,male,cebc4b3995,28.0,white,SCHP,vehicular,56-10-225 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Radar Triggered,Citation,M,White,---------,Other Violations,NaN,White
1,2,2007-10-17,SC Primary 86,NaN,NaN,Anderson County,17.0,white,male,ef3225d61e,34.0,white,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,M,White,NaN,NaN,NaN,White
2,3,2007-03-20,SC Primary 11,NaN,NaN,Greenville County,68.0,white,male,808e789e5f,44.0,white,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,White,---------,Speeding or Too Fast For Conditions (-- m.p.h....,NaN,White
3,4,2007-07-02,US Prmary 29,NaN,NaN,Spartanburg County,27.0,black,female,5df6d7974a,31.0,white,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Radar Triggered,Citation,F,Black,---------,Speeding or Too Fast For Conditions (----- m.p...,NaN,White
4,5,2008-11-08,Interstate 20,34.20619,-79.99261,Darlington County,29.0,white,male,35d5dd2f85,32.0,black,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,White,---------,Speeding or Too Fast For Conditions (----- m.p...,NaN,Black


In [8]:
print(df.isnull().sum())

raw_row_number              0
date                        3
location                    1
lat                   6880144
lng                   6880940
county_name               974
subject_age              4285
subject_race                3
subject_sex                 3
officer_id_hash             0
officer_age             43451
officer_race             4692
department_id             121
type                       36
violation             3027892
arrest_made                 5
citation_issued             2
outcome               3027861
contraband_found      8733656
contraband_drugs      8734151
contraband_weapons    8734151
contraband_alcohol    8983810
contraband_other      8792932
search_conducted            0
search_person               0
search_vehicle              0
reason_for_stop            36
raw_contact_type            2
raw_sex                     1
raw_race                    1
raw_sectionnum        3028434
raw_offensecode       3027916
raw_contrabanddesc    8923945
raw_office

In [9]:
# filling in any null values
df = df.fillna('NaN')

In [10]:
# don't need latitude, longitude or 'raw_sectionnum'
df = df.drop(['lat', 'lng', 'raw_sectionnum'], axis=1)

In [16]:
# checking the types of variables we have
df.dtypes

raw_row_number         int64
date                  object
location              object
county_name           object
subject_age           object
subject_race          object
subject_sex           object
officer_id_hash       object
officer_age           object
officer_race          object
department_id         object
type                  object
violation             object
arrest_made           object
citation_issued       object
outcome               object
contraband_found      object
contraband_drugs      object
contraband_weapons    object
contraband_alcohol    object
contraband_other      object
search_conducted        bool
search_person           bool
search_vehicle          bool
reason_for_stop       object
raw_contact_type      object
raw_sex               object
raw_race              object
raw_offensecode       object
raw_contrabanddesc    object
raw_officer_race      object
dtype: object

In [11]:
# let's break up the 'date' column into individual 'year', 'month' and 'day' columns

# first, we'll need to conver 'date' to datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [12]:
# let's check to make sure it worked

df['date'].head()

0   2006-02-12
1   2007-10-17
2   2007-03-20
3   2007-07-02
4   2008-11-08
Name: date, dtype: datetime64[ns]

In [13]:
# and then we can actually break up the columns
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [20]:
# check: is it there? Yes!
print(df.shape)
df.head()

(8983810, 37)


,raw_row_number,date,location,lat,lng,county_name,subject_age,subject_race,subject_sex,officer_id_hash,officer_age,officer_race,department_id,type,violation,arrest_made,citation_issued,outcome,contraband_found,contraband_drugs,contraband_weapons,contraband_alcohol,contraband_other,search_conducted,search_person,search_vehicle,reason_for_stop,raw_contact_type,raw_sex,raw_race,raw_sectionnum,raw_offensecode,raw_contrabanddesc,raw_officer_race,year,month,day
0,1,2006-02-12,US Prmary 17,33.1995,-79.3927,Georgetown County,33,white,male,cebc4b3995,28,white,SCHP,vehicular,56-10-225 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Radar Triggered,Citation,M,White,---------,Other Violations,NaN,White,2006.0,2.0,12.0
1,2,2007-10-17,SC Primary 86,NaN,NaN,Anderson County,17,white,male,ef3225d61e,34,white,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,M,White,NaN,NaN,NaN,White,2007.0,10.0,17.0
2,3,2007-03-20,SC Primary 11,NaN,NaN,Greenville County,68,white,male,808e789e5f,44,white,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,White,---------,Speeding or Too Fast For Conditions (-- m.p.h....,NaN,White,2007.0,3.0,20.0
3,4,2007-07-02,US Prmary 29,NaN,NaN,Spartanburg County,27,black,female,5df6d7974a,31,white,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Radar Triggered,Citation,F,Black,---------,Speeding or Too Fast For Conditions (----- m.p...,NaN,White,2007.0,7.0,2.0
4,5,2008-11-08,Interstate 20,34.2062,-79.9926,Darlington County,29,white,male,35d5dd2f85,32,black,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,White,---------,Speeding or Too Fast For Conditions (----- m.p...,NaN,Black,2008.0,11.0,8.0


In [133]:
df['reason_for_stop'].value_counts()

Violation Observed     6763255
Radar Triggered        1476697
Collision               498214
SW Enforcement           86852
Inspection               74783
Pedestrian               44662
Inquiry                  25645
Bolo Follow Up            5483
Incident                  4557
Suspicious Activity       3624
NaN                         36
-                            2
Name: reason_for_stop, dtype: int64

In [41]:
# checking all racial demographics
df['subject_race'].value_counts()

white       5564909
black       2942644
hispanic     376969
other         97123
unknown        2162
NaN               3
Name: subject_race, dtype: int64

In [61]:
# and we'll convert the year into a string so that we can filter later on
# df['year'] = df['year'].astype(str).astype(float)

In [131]:
df['officer_id_hash'].nunique()

1827

In [10]:
# the column 'officer_id_hash' lists all state patrol officers (who made arrests?)
# let's see who's been making the most arrests
# (the number of times that a hash id appears equals the number of arrests)

# woooooooow, officer 979aad30f8  made 28,884 arrests

df['officer_id_hash'].value_counts()

979aad30f8    28884
d8bde075a9    25722
02bcb96498    24245
2bce550242    24204
147e11d643    24021
              ...  
039702c03a        1
8112924839        1
0ba29539a8        1
156e828756        1
cab2c79b14        1
Name: officer_id_hash, Length: 1827, dtype: int64

In [14]:
# we're going to examine who the officer with the most arrests is, and as much info about him as possible.
# we'll call him Officer X
# what we've found out so far:
  # the officer is white
  # his arrests are predominantly white; he has arrested ~3,000 more white people than Black people
  # his arrests are predominantly male; he has arrested ~8,000 more men than women
officer1 = df['officer_id_hash'].str.contains("979aad30f8")
officer_main = df[officer1]
print(officer_main.shape)
officer_main.head()

(28884, 34)


,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,officer_id_hash,officer_age,officer_race,department_id,type,violation,arrest_made,citation_issued,outcome,contraband_found,contraband_drugs,contraband_weapons,contraband_alcohol,contraband_other,search_conducted,search_person,search_vehicle,reason_for_stop,raw_contact_type,raw_sex,raw_race,raw_offensecode,raw_contrabanddesc,raw_officer_race,year,month,day
398,399,2011-02-26,Interstate 20,Darlington County,20,black,female,979aad30f8,35,white,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,F,Black,Speeding or Too Fast For Conditions (-- m.p.h....,NaN,White,2011.0,2.0,26.0
1065,1066,2016-05-26,Interstate 20,Lee County,27,black,male,979aad30f8,41,white,SCHP,vehicular,56-5-6520 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,Black,Other Violations,NaN,White,2016.0,5.0,26.0
1270,1271,2012-07-09,US Prmary 76,Sumter County,31,white,female,979aad30f8,37,white,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,F,White,NaN,NaN,White,2012.0,7.0,9.0
1350,1351,2014-05-22,Interstate 20,Lee County,32,black,male,979aad30f8,39,white,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,M,Black,NaN,NaN,White,2014.0,5.0,22.0
1540,1541,2005-06-19,Interstate 20,Lee County,28,white,male,979aad30f8,30,white,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,White,Speeding or Too Fast For Conditions (-- m.p.h....,NaN,White,2005.0,6.0,19.0


In [110]:
# where has he made arrests?

print("Total number of unique arrest counties: ", officer_main['county_name'].nunique())
print("Total number of unique arrest locations: ", officer_main['location'].nunique())

Total number of unique arrest counties:  11
Total number of unique arrest locations:  357


In [111]:
# Where did he make the most arrests?
  # Interstate 20
officer_main['location'].value_counts()

Interstate 20            9220
US Prmary 15             3108
US Prmary 1              2129
SC Primary 151           1967
US Prmary 521             961
                         ... 
Secondary 199               1
County two notch road       1
Secondary 482               1
Secondary 306               1
Secondary 545               1
Name: location, Length: 357, dtype: int64

In [112]:
# In what counties did he make the most arrests?
  # Darlington
  # Lee
  # Kershaw
  # Marlboro
  # Sumter

officer_main['county_name'].value_counts()

Darlington County    12232
Lee County            7312
Kershaw County        6848
Marlboro County       1494
Sumter County          658
Richland County        112
Horry County           103
Dillion County          61
Marion County           57
Claredon County          6
NaN                      1
Name: county_name, dtype: int64

In [113]:
# his current age
# Officer X's last arrest was made in 2016, when he is recorded as 41
# So, in 2020, he is around 45 years old (of course, this depends on his birthday)
officer_main['officer_age'].sort_values()

4430537    29
8461555    29
7261413    29
5457556    29
2332682    29
           ..
4484958    41
8077803    41
3415725    41
3974742    41
6097643    41
Name: officer_age, Length: 28884, dtype: object

In [69]:
########## Analyzing arrest demographics throughout Officer X's career

In [114]:
# what's the primary race this officer has arrested during the time period?
  # white
officer_main['subject_race'].value_counts()

white       16404
black       11588
hispanic      490
other         402
Name: subject_race, dtype: int64

In [115]:
# what's the primary race this officer has arrested during the time period
officer_main['subject_sex'].value_counts()

male      18710
female    10174
Name: subject_sex, dtype: int64

In [117]:
# what about the intersection of race and gender?

# white men and women
white2 = officer_main['subject_race'].str.contains("white")
white = officer_main[white2]
white['subject_sex'].value_counts()

male      10958
female     5446
Name: subject_sex, dtype: int64

In [74]:
# black men and women
black2 = officer_main['subject_race'].str.contains("black")
black = officer_main[black2]
black['subject_sex'].value_counts()

male      7025
female    4563
Name: subject_sex, dtype: int64

In [75]:
# hispanic men and women
hisp2 = officer_main['subject_race'].str.contains("hispanic")
hisp = officer_main[hisp2]
hisp['subject_sex'].value_counts()

male      421
female     69
Name: subject_sex, dtype: int64

In [76]:
# other men and women
other2 = officer_main['subject_race'].str.contains("other")
other = officer_main[other2]
other['subject_sex'].value_counts()

male      306
female     96
Name: subject_sex, dtype: int64

In [77]:
!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

/usr/local/bin/orca: Text file busy
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgtk2.0-0 is already the newest version (2.24.32-1ubuntu1).
libgconf-2-4 is already the newest version (3.2.6-4ubuntu1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# plotting the data...how do they compare?

import plotly.graph_objects as go

race=['White', 'Black', 'Hispanic', 'Other']

fig = go.Figure(data=[
    go.Bar(name='<b>Men</b>', x=race, y=[10958, 7025, 421, 306], 
           marker_color='#2980B9'),
    go.Bar(name='<b>Women</b>', x=race, y=[5446, 4563, 69, 96],
          marker_color='#5DADE2')
])

# Change the bar mode
fig.update_layout(barmode='group', title_text="<b>Trooper X's arrests, by race and gender (2005-2016)</b>",
                  xaxis_title="<b>Race</b>",
                  yaxis_title="<b>Number of arrests</b>",
                  font=dict(family="Courier New, monospace", size=14,
                  color="Black"), legend=dict(yanchor="top",
                                              y=0.99, xanchor="left",
                                              x=0.7, font=dict(family="Courier",
                                                               size=30, color="black")))

# gotta save my plot!
fig.write_image("bw_graph1.png")
fig.show()



In [79]:
from google.colab import files
files.download('bw_graph1.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# plotting the sex data
fig, ax = plt.subplots()

x = ['Male', 'Female']
y = [18710, 10174]

ax.bar(x, y)


ax.set_title("Officer X's total arrests, by gender", fontsize=16, fontweight="bold")
ax.set_xlabel("Gender", fontsize=12, fontweight="bold")
ax.set_ylabel("Number of arrests (thousands)", fontsize=11, fontweight="bold")

ax.set_xticklabels(["Male", "Female"], fontsize=10)
ax.set_yticklabels(["0", "2.5", "5", "7.5", "10", "12.5", "15", "17.5"])

plt.show()

In [ ]:
######### comparing Officer X arrests to state average

In [15]:
# the number of rows in the datadrame tells us the total number of arrests from 2005-2010
df.shape

(8983810, 34)

In [13]:
# the number of unique officer hash ids tells us how many officers there are, 
# which we can use as the len when finding the state arrests mean
df['officer_id_hash'].nunique()

1827

In [16]:
# state arrests mean
  # the mean number of arrests is around 4917
8983810 / 1827

4917.246852764094

In [81]:
# when did he make the most arrests?
  # 2010
officer_main['year'].value_counts()

2010.0    3133
2012.0    2906
2011.0    2744
2009.0    2499
2013.0    2480
2006.0    2470
2015.0    2401
2014.0    2227
2008.0    2190
2007.0    2158
2005.0    2137
2016.0    1539
Name: year, dtype: int64

In [84]:
######### comparing Officer X's 2010 arrests to the state average

In [15]:
x2010 = officer_main['year']==2010.0
officer_main2010 = officer_main[x2010]
officer_main2010

,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,officer_id_hash,officer_age,officer_race,department_id,type,violation,arrest_made,citation_issued,outcome,contraband_found,contraband_drugs,contraband_weapons,contraband_alcohol,contraband_other,search_conducted,search_person,search_vehicle,reason_for_stop,raw_contact_type,raw_sex,raw_race,raw_offensecode,raw_contrabanddesc,raw_officer_race,year,month,day
2710,2711,2010-09-12,Interstate 20,Darlington County,26,white,male,979aad30f8,35,white,SCHP,vehicular,56-10-225 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,White,Other Violations,NaN,White,2010.0,9.0,12.0
6076,6077,2010-09-20,US Prmary 15,Darlington County,108,white,female,979aad30f8,35,white,SCHP,vehicular,56-5-6520 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,F,White,Other Violations,NaN,White,2010.0,9.0,20.0
8282,8283,2010-06-19,SC Primary 38,Marlboro County,21,white,male,979aad30f8,35,white,SCHP,vehicular,56-5-6520 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,White,Other Violations,NaN,White,2010.0,6.0,19.0
8480,8481,2010-08-15,Interstate 20,Darlington County,36,black,female,979aad30f8,35,white,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,F,Black,Speeding or Too Fast For Conditions (----- m.p...,NaN,White,2010.0,8.0,15.0
12276,12277,2010-09-06,Interstate 20,Darlington County,20,white,male,979aad30f8,35,white,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,White,Speeding or Too Fast For Conditions (----- m.p...,NaN,White,2010.0,9.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8965541,8965542,2010-12-31,Secondary 165,Darlington County,53,white,male,979aad30f8,35,white,SCHP,vehicular,56-5-6520 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,White,Other Violations,NaN,White,2010.0,12.0,31.0
8971458,8971459,2010-10-13,Secondary 23,Darlington County,19,white,male,979aad30f8,35,white,SCHP,vehicular,56-1-460 Driving Under Suspension,True,True,arrest,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,White,Driving Under Suspension,NaN,White,2010.0,10.0,13.0
8975977,8975978,2010-12-08,US Prmary 15,Darlington County,61,black,male,979aad30f8,35,white,SCHP,vehicular,56-5-6520 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,Black,Other Violations,NaN,White,2010.0,12.0,8.0
8979614,8979615,2010-05-04,US Prmary 15,Darlington County,29,white,male,979aad30f8,35,white,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,M,White,NaN,NaN,White,2010.0,5.0,4.0


In [136]:
officer_main['subject_race'].value_counts()

white       16404
black       11588
hispanic      490
other         402
Name: subject_race, dtype: int64

In [27]:
# total 2010 arrests
# number of rows tells us the sum

is2010 =  df['year']==2010.0
y2010 = df[is2010]
print(y2010.shape)
y2010

(852048, 34)


,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,officer_id_hash,officer_age,officer_race,department_id,type,violation,arrest_made,citation_issued,outcome,contraband_found,contraband_drugs,contraband_weapons,contraband_alcohol,contraband_other,search_conducted,search_person,search_vehicle,reason_for_stop,raw_contact_type,raw_sex,raw_race,raw_offensecode,raw_contrabanddesc,raw_officer_race,year,month,day
6,7,2010-04-21,US Prmary 176,Calhoun County,20,black,male,56ace3c651,29,white,SCHP,vehicular,56-5-6520 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,Black,Other Violations,NaN,White,2010.0,4.0,21.0
7,8,2010-05-03,Interstate 77,York County,24,white,male,d4c160de5b,29,white,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,White,Speeding or Too Fast For Conditions (----- m.p...,NaN,White,2010.0,5.0,3.0
29,30,2010-06-18,SC Primary 151,Darlington County,37,black,male,2f015e0f4d,24,black,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,M,Black,NaN,NaN,Black,2010.0,6.0,18.0
76,77,2010-03-08,Secondary 80,Beaufort County,42,black,male,3e67e71a09,44,white,SCHP,vehicular,56-5-6520 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,Black,Other Violations,NaN,White,2010.0,3.0,8.0
97,98,2010-03-05,SC Primary 4,Orangeburg County,29,white,male,9310b80bf7,28,black,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,M,White,NaN,NaN,Black,2010.0,3.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8983743,8983744,2010-08-25,SC Primary 230,Aiken County,34,black,female,9f40682bbf,33,white,SCHP,vehicular,56-5-950 Disregarding Sign or Signal,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,F,Black,Disregarding Sign or Signal,NaN,White,2010.0,8.0,25.0
8983759,8983760,2010-03-13,Interstate 20,Aiken County,23,black,female,b784e33e83,24,black,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Radar Triggered,Citation,F,Black,Speeding or Too Fast For Conditions (----- m.p...,NaN,Black,2010.0,3.0,13.0
8983762,8983763,2010-03-28,Interstate 20,Kershaw County,25,black,male,32fbfd02ac,26,white,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,Black,Speeding or Too Fast For Conditions (-- m.p.h....,NaN,White,2010.0,3.0,28.0
8983774,8983775,2010-03-04,US Prmary 521,Kershaw County,79,white,female,286f1062d3,36,white,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,F,White,NaN,NaN,White,2010.0,3.0,4.0


In [19]:
# now we're going to do a two-sample t-test to test the Officer X 2010 arrest racial demogaphics to the statewide 2010 racial demographics
# Null: Officer X's 2010 racial arrest demographics are equal to the statewide 2010 racial arrest demographics
# Alternative: Officer X's 2010 racial arrest demographics have a higher proportion of white people arrested than the  statewide 2010 racial arrest demographics
# ttest2010 = stats.ttest_ind(y2010['subject_race'], officer_main2010['subject_race'])

KeyboardInterrupt: ignored

In [ ]:
# what is his earliest arrest date?
  # 2005
officer_main['date'].sort_values()

In [134]:
# what are Officer X's top five reasons for arresting someone?
# turns out, he's made arrests for three different violations:

officer_main['reason_for_stop'].value_counts()

Violation Observed    28720
Pedestrian              142
Bolo Follow Up           22
Name: reason_for_stop, dtype: int64

In [ ]:
# and which years had the most arrests?
# first, we've got to create a new column for the years

In [ ]:
######### comparing Officer X to another officer chosen through random sampling

In [44]:
# but we also want to compare him against another officer
# so we'll draw a random sample
# our random officer has the hash id: 576e3a5317
officer_sample = y2010['officer_id_hash'].sample()
print("Random Officer: ", officer_sample)

Random Officer:  5405522    6b458002e3
Name: officer_id_hash, dtype: object


In [45]:
# we're going to examine who the officer with the most arrests is, and as much info about him as possible.
# we'll call him Officer R, for "random"
# what we've found out so far:
  # the officer is white
  # his arrests are predominantly white; he has arrested ~3,000 more white people than Black people
  # his arrests are predominantly male; he has arrested ~8,000 more men than women
off_samp = y2010['officer_id_hash'].str.contains("6b458002e3")
officer_random = y2010[off_samp]
print(officer_random.shape)
officer_random.head()

(1579, 34)


,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,officer_id_hash,officer_age,officer_race,department_id,type,violation,arrest_made,citation_issued,outcome,contraband_found,contraband_drugs,contraband_weapons,contraband_alcohol,contraband_other,search_conducted,search_person,search_vehicle,reason_for_stop,raw_contact_type,raw_sex,raw_race,raw_offensecode,raw_contrabanddesc,raw_officer_race,year,month,day
4598,4599,2010-05-17,US Prmary 176,Richland County,27,black,male,6b458002e3,26,white,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,M,Black,NaN,NaN,White,2010.0,5.0,17.0
7137,7138,2010-06-28,Secondary 31,Richland County,43,white,female,6b458002e3,26,white,SCHP,vehicular,56-5-6520 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,F,White,Other Violations,NaN,White,2010.0,6.0,28.0
14595,14596,2010-10-04,Interstate 77,Richland County,27,white,male,6b458002e3,26,white,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,M,White,NaN,NaN,White,2010.0,10.0,4.0
14596,14597,2010-04-09,Secondary 1036,Richland County,19,black,female,6b458002e3,26,white,SCHP,vehicular,56-5-2320 No Right of Way,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Collision,Citation,F,Black,No Right of Way,NaN,White,2010.0,4.0,9.0
21404,21405,2010-05-13,US Prmary 176,Richland County,25,black,female,6b458002e3,26,white,SCHP,vehicular,56-5-6520 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,F,Black,Other Violations,NaN,White,2010.0,5.0,13.0


In [32]:
# Where did he make the most arrests?
  # Interstate 20
officer_random['location'].value_counts()

US Prmary 17             1052
SC Primary 544            645
SC Primary 707            563
US Prmary 501             271
Secondary 1240             81
                         ... 
Secondary 707               1
County breezewood dr.       1
Secondary 66                1
County cox ferry rd         1
County HOW. PKWY            1
Name: location, Length: 191, dtype: int64

In [46]:
# his current age
# Officer X's last arrest was made in 2016, when he is recorded as 41
# So, in 2020, he is around 45 years old (of course, this depends on his birthday)
officer_random['officer_age'].sort_values()

4598       26
5794469    26
5787100    26
5779069    26
5766821    26
           ..
4467323    27
1917990    27
4434849    27
3225879    27
6541345    27
Name: officer_age, Length: 1579, dtype: object

In [47]:
# what's the primary race this officer has arrested during the time period?
  # white
officer_random['subject_race'].value_counts()

black       877
white       598
hispanic     65
other        39
Name: subject_race, dtype: int64

In [48]:
# what's the primary race this officer has arrested during the time period
officer_random['subject_sex'].value_counts()

male      988
female    591
Name: subject_sex, dtype: int64

In [49]:
sex_count_or = officer_random['subject_sex'].value_counts()
sex_count_or

male      988
female    591
Name: subject_sex, dtype: int64

In [50]:
# what about the intersection of race and gender?

# white men and women
rwhite2 = officer_random['subject_race'].str.contains("white")
rwhite = officer_random[rwhite2]
rwhite['subject_sex'].value_counts()

male      379
female    219
Name: subject_sex, dtype: int64

In [51]:
# black men and women
rblack2 = officer_random['subject_race'].str.contains("black")
rblack = officer_random[rblack2]
rblack['subject_sex'].value_counts()

male      528
female    349
Name: subject_sex, dtype: int64

In [52]:
# hispanic men and women
rhisp2 = officer_random['subject_race'].str.contains("hispanic")
rhisp = officer_random[rhisp2]
rhisp['subject_sex'].value_counts()

male      52
female    13
Name: subject_sex, dtype: int64

In [53]:
# other men and women
rother2 = officer_random['subject_race'].str.contains("other")
rother = officer_random[rother2]
rother['subject_sex'].value_counts()

male      29
female    10
Name: subject_sex, dtype: int64

In [63]:
# need to find Officer R's 2010 data
r2010 = officer_random['year']==2010.0
officer_random2010 = officer_random[is2010]
print(officer_random2010.shape)
officer_random2010

(1579, 34)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,officer_id_hash,officer_age,officer_race,department_id,type,violation,arrest_made,citation_issued,outcome,contraband_found,contraband_drugs,contraband_weapons,contraband_alcohol,contraband_other,search_conducted,search_person,search_vehicle,reason_for_stop,raw_contact_type,raw_sex,raw_race,raw_offensecode,raw_contrabanddesc,raw_officer_race,year,month,day
4598,4599,2010-05-17,US Prmary 176,Richland County,27,black,male,6b458002e3,26,white,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,M,Black,NaN,NaN,White,2010.0,5.0,17.0
7137,7138,2010-06-28,Secondary 31,Richland County,43,white,female,6b458002e3,26,white,SCHP,vehicular,56-5-6520 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,F,White,Other Violations,NaN,White,2010.0,6.0,28.0
14595,14596,2010-10-04,Interstate 77,Richland County,27,white,male,6b458002e3,26,white,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,M,White,NaN,NaN,White,2010.0,10.0,4.0
14596,14597,2010-04-09,Secondary 1036,Richland County,19,black,female,6b458002e3,26,white,SCHP,vehicular,56-5-2320 No Right of Way,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Collision,Citation,F,Black,No Right of Way,NaN,White,2010.0,4.0,9.0
21404,21405,2010-05-13,US Prmary 176,Richland County,25,black,female,6b458002e3,26,white,SCHP,vehicular,56-5-6520 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,F,Black,Other Violations,NaN,White,2010.0,5.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8927566,8927567,2010-08-27,Interstate 77,Richland County,26,white,male,6b458002e3,26,white,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,White,Speeding or Too Fast For Conditions (-- m.p.h....,NaN,White,2010.0,8.0,27.0
8930378,8930379,2010-09-29,US Prmary 176,Richland County,66,white,female,6b458002e3,26,white,SCHP,vehicular,56-5-6520 Other Violations,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,F,White,Other Violations,NaN,White,2010.0,9.0,29.0
8945919,8945920,2010-10-23,Interstate 26,Richland County,46,white,female,6b458002e3,26,white,SCHP,vehicular,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Public Contact,F,White,NaN,NaN,White,2010.0,10.0,23.0
8962533,8962534,2010-08-23,US Prmary 176,Richland County,72,black,male,6b458002e3,26,white,SCHP,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,NaN,NaN,NaN,NaN,NaN,False,False,False,Violation Observed,Citation,M,Black,Speeding or Too Fast For Conditions (-- m.p.h....,NaN,White,2010.0,8.0,23.0


In [ ]:
######## comparing 2010 gender/racial arrest demographics of Officer X and the state

In [79]:
print("Officer X: \n", officer_main2010['subject_race'].value_counts())
print("Statewide: \n", y2010['subject_race'].value_counts())

Officer X: 
 white       1885
black       1166
other         45
hispanic      37
Name: subject_race, dtype: int64
Statewide: 
 white       539510
black       268967
hispanic     34435
other         8901
unknown        235
Name: subject_race, dtype: int64


In [120]:
# len statewide 2010 data
y2010['officer_id_hash'].nunique()

983

In [82]:
print(officer_main2010.shape)
print(y2010.shape)

(3133, 34)
(1579, 34)
(852048, 34)


In [121]:
# mean of statewide 2010 data
852048 / 983

866.7833163784334

In [129]:
# Gotta plot my findings!

fig = go.Figure(data=go.Scatter(
    x=[0, 1],
    y=[3133, 866.78],
    mode='markers',
    marker=dict(size=[80, 80],
                color=['#2980B9', 'yellow'])
))

fig.update_layout(title_text="<b>2010 arrest comparisons</b>",
                  xaxis_title="<b>Population</b>",
                  yaxis_title="<b>Arrests average</b>",
                  font=dict(family="Courier New, monospace", size=14,
                  color="Black"), legend=dict(yanchor="top",
                                              y=0.1, xanchor="left",
                                              x=0.7, font=dict(family="Courier",
                                                               size=30, color="black")),
                  xaxis = dict(tickmode = 'array', tickvals = [0, 1],
                               ticktext = ['Officer X', 'Statewide']))
          
#)
fig.write_image("bw_graph2.png")
                  
fig.show()

In [130]:
from google.colab import files
files.download('bw_graph2.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>